In [ ]:
import sys

sys.path.append('..')

from torch.utils.data import DataLoader
import pandas as pd
from pathlib import Path
import numpy as np

from src.dataset import HumanPosesDataset
from sklearn.model_selection import train_test_split
import torch

# Ансамбль

## Датасет для валидации

In [ ]:
from torchvision import transforms

mean = [0.4638, 0.4522, 0.4148]
std = [0.2222, 0.2198, 0.2176]

tta_transforms = [
    transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ]),
    transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.RandomHorizontalFlip(p=1.0),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ]),
    transforms.Compose([
        transforms.Resize(256),
        transforms.RandomResizedCrop(224, scale=(0.9, 1.0)),
        transforms.RandomHorizontalFlip(p=1.0),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ]),
]

In [ ]:
CSV_PATH = Path("../data/human_poses_data/train_answers.csv")
TRAIN_DIR = Path("../data/human_poses_data/img_train")

df = pd.read_csv(CSV_PATH)

train_ids, val_ids = train_test_split(
    df['img_id'].values,
    test_size=0.2,
    stratify=df['target_feature'],
    random_state=42
)

val_df = df[df['img_id'].isin(val_ids)].reset_index(drop=True)


val_dataset = HumanPosesDataset(
    data_df=val_df,
    img_dir=TRAIN_DIR,
    transform=None, #только в этом ноутбуке
)

def pil_collate(batch):
    images, labels = zip(*batch)
    return list(images), list(labels)

val_loader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=0,
    collate_fn=pil_collate,
    pin_memory=True
)

print(f"Validation dataset size: {len(val_dataset)}")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")

## Загрузка моделей и весов

In [ ]:
from src.models.tinyvit import TinyViT
from src.models.enet import EfficientNetV2
from src.models.convnextv2 import ConvNeXtV2
from src.models.miniconvnext import MiniConvNeXt
from src.utils import load_best_model

num_classes = 16

tiny_vit_ssl = TinyViT(in_chans=3, num_classes=num_classes, model_size='11M').to(device)
load_best_model(tiny_vit_ssl, '../best_models/ssl_vit_1_5_best.pth', device)

tiny_vit_no_ssl = TinyViT(in_chans=3, model_size='11M', num_classes=num_classes).to(device)
load_best_model(tiny_vit_no_ssl, '../best_models/my_vit_4_best.pth', device)

enet_v2_m = EfficientNetV2(variant='M', num_classes=num_classes).to(device)
load_best_model(enet_v2_m, '../best_models/enet_1_5_best.pth', device)

convnextv2_small = ConvNeXtV2(num_classes=num_classes, variant='small').to(device)
load_best_model(convnextv2_small, '../best_models/convnextv2_small_3_best.pth', device)

convnextv2_base = ConvNeXtV2(num_classes=num_classes, variant='base').to(device)
load_best_model(convnextv2_base, '../best_models/convnextv2_base_3_best.pth', device)

miniconvnext = MiniConvNeXt(num_classes=num_classes).to(device)
load_best_model(miniconvnext, '../best_models/convnext_last.pth', device)

In [ ]:
def validate_all_models(models, val_loader, tta_transforms, true_labels, device):
    from src.ensemble import WeightedEnsemble

    scores = {}

    for name, model in models.items():
        print(f"\n🔍 Validating model: {name}")

        ensemble = WeightedEnsemble({name: 1.0})
        predictions, _ = ensemble.predict_from_models(
            models={name: model},
            dataloader=val_loader,
            device=device,
            tta_transforms=tta_transforms
        )
        f1 = ensemble.validate(predictions, true_labels)
        scores[name] = f1

    print("\nAll F1-scores:")
    for name, score in scores.items():
        print(f"{name}: {score:.4f}")

    return scores

In [ ]:
models = {
    "tiny_vit_ssl": tiny_vit_ssl,
    "tiny_vit_no_ssl": tiny_vit_no_ssl,
    "enet_v2_m": enet_v2_m,
    "convnextv2_small": convnextv2_small,
    "convnextv2_base": convnextv2_base,
    "miniconvnext": miniconvnext
}

true_labels = list(val_dataset.labels)

validate_all_models(models, val_loader, tta_transforms, true_labels, device)

F1-scores: <br>
tiny_vit_ssl: 0.7252 <br>
tiny_vit_no_ssl: 0.7207 <br>
enet_v2_m: 0.7147 <br>
convnextv2_small: 0.7188 <br>
convnextv2_base: 0.6958 <br>
miniconvnext: 0.6969 <br>

## Optuna

In [ ]:
from src.ensemble import WeightedEnsemble

ensemble = WeightedEnsemble({name: 1.0 for name in models})
predictions, _ = ensemble.predict_from_models(
    models=models,
    dataloader=val_loader,
    device=device,
    tta_transforms=tta_transforms
)

In [ ]:
np.savez("val_logits.npz", **predictions)
true_labels = list(val_dataset.labels)
np.save("true_labels.npy", true_labels)

In [ ]:
import numpy as np
import optuna

logits_data = np.load("val_logits.npz")

logits_dict = {k: v for k, v in logits_data.items()}
model_names = list(logits_dict.keys())

true_labels = np.load("true_labels.npy")

num_classes = next(iter(logits_dict.values())).shape[1]


def objective(trial):
    weights = np.array([trial.suggest_float(name, 0.0, 1.0) for name in model_names])
    weights /= weights.sum() + 1e-8

    logits = sum(w * logits_dict[name] for w, name in zip(weights, model_names))
    preds = np.argmax(logits, axis=1)
    return f1_score(true_labels, preds, average="macro")


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

study.best_params, study.best_value

In [ ]:
from src.ensemble import WeightedEnsemble

ensemble = WeightedEnsemble({
    'tiny_vit_ssl': 0.18659441736954463,
    'tiny_vit_no_ssl': 0.24930515606163378,
    'enet_v2_m': 0.32824274496866923,
    'convnextv2_small': 0.4704414186510291,
    'convnextv2_base': 0.185445131248959,
    'miniconvnext': 0.38462473460682
})

models = {"tiny_vit_no_ssl": tiny_vit_no_ssl,
          "tiny_vit_ssl": tiny_vit_ssl,
          'enet_v2_m': enet_v2_m,
          'convnextv2_small': convnextv2_small,
          'convnextv2_base': convnextv2_base,
          'miniconvnext': miniconvnext}

## Валидация

In [ ]:
true_labels = val_dataset.labels.tolist()

predictions, _ = ensemble.predict_from_models(
    models=models,
    dataloader=val_loader,
    device=device,
    tta_transforms=tta_transforms
)

ensemble.validate(predictions, true_labels)

## Сабмит

In [ ]:
from src.ensemble import TestDataset

TEST_DIR = Path("../data/human_poses_data/img_test")

test_image_paths = list(TEST_DIR.glob("*.jpg"))

test_ids = [int(p.stem) for p in test_image_paths]

test_dataset = TestDataset(test_image_paths, test_ids)

def pil_collate(batch):
    images, ids = zip(*batch)
    return list(images), list(ids)

test_loader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=0,
    collate_fn=pil_collate,
    pin_memory=True
)

In [ ]:
predictions, image_ids = ensemble.predict_from_models(
    models=models,
    dataloader=test_loader,
    device=device,
    tta_transforms=tta_transforms
)

ensemble.make_submission(
    predictions=predictions,
    image_ids=image_ids,
    index_to_class=val_dataset.index_to_class,
    output_path="submission.csv"
)

# Стекинг

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from scipy.special import softmax

logits_dict = np.load("val_logits.npz")
y = np.load("true_labels.npy")

X_raw = np.concatenate([logits_dict[k] for k in logits_dict], axis=1)

def meta_features(logits):
    probs = softmax(logits, axis=1)
    entropy = -np.sum(probs * np.log(probs + 1e-8), axis=1, keepdims=True)
    confidence = np.max(probs, axis=1, keepdims=True)
    sorted_probs = -np.sort(-probs, axis=1)
    margin = (sorted_probs[:, 0] - sorted_probs[:, 1]).reshape(-1, 1)
    return np.concatenate([entropy, confidence, margin], axis=1)

meta_feats = np.concatenate([meta_features(logits_dict[k]) for k in logits_dict], axis=1)
X = np.concatenate([X_raw, meta_feats], axis=1)

n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
oof_preds_cat = np.zeros((len(y), num_classes))
oof_preds_lgb = np.zeros((len(y), num_classes))

for train_idx, val_idx in skf.split(X, y):
    X_train, y_train = X[train_idx], y[train_idx]
    X_val = X[val_idx]

    cat = CatBoostClassifier(iterations=500, learning_rate=0.05, depth=4, l2_leaf_reg=10,
                             early_stopping_rounds=20, verbose=False)
    lgb = LGBMClassifier(n_estimators=500, learning_rate=0.05, max_depth=4, reg_lambda=10)

    cat.fit(X_train, y_train)
    lgb.fit(X_train, y_train)

    oof_preds_cat[val_idx] = cat.predict_proba(X_val)
    oof_preds_lgb[val_idx] = lgb.predict_proba(X_val)

optuna_weights = {
    'tiny_vit_ssl': 0.18659441736594463,
    'tiny_vit_no_ssl': 0.24903515606163378,
    'enet_v2_m': 0.3282474648666293,
    'convnextv2_small': 0.4790441486510921,
    'convnextv2_base': 0.185445312408959,
    'miniconvnext': 0.38462473466082
}

norm = sum(optuna_weights.values())
optuna_weights = {k: v / norm for k, v in optuna_weights.items()}

oof_preds_opt = np.zeros_like(oof_preds_cat)
for name, weight in optuna_weights.items():
    oof_preds_opt += weight * logits_dict[name]

oof_blend = 0.4 * oof_preds_cat + 0.3 * oof_preds_lgb + 0.3 * oof_preds_opt
oof_pred_labels = np.argmax(oof_blend, axis=1)
f1 = f1_score(y, oof_pred_labels, average="macro")
print(f"\nFINAL OOF F1: {f1:.5f}")

In [ ]:
cat_final = CatBoostClassifier(iterations=500, learning_rate=0.05, depth=4, l2_leaf_reg=10,
                               early_stopping_rounds=20, verbose=False)
lgb_final = LGBMClassifier(n_estimators=500, learning_rate=0.05, max_depth=4, reg_lambda=10)
cat_final.fit(X, y)
lgb_final.fit(X, y)

In [ ]:
test_logits = np.load("test_logits.npz")
test_ids = np.load("test_ids.npy")
X_test_raw = np.concatenate([test_logits[k] for k in test_logits], axis=1)
meta_feats_test = np.concatenate([meta_features(test_logits[k]) for k in test_logits], axis=1)
X_test = np.concatenate([X_test_raw, meta_feats_test], axis=1)

test_preds_opt = np.zeros((len(test_ids), num_classes))
for name, weight in optuna_weights.items():
    test_preds_opt += weight * test_logits[name]

cat_probs = cat_final.predict_proba(X_test)
lgb_probs = lgb_final.predict_proba(X_test)
final_probs = 0.4 * cat_probs + 0.3 * lgb_probs + 0.3 * test_preds_opt
y_pred = np.argmax(final_probs, axis=1)

index_to_class = val_dataset.index_to_class
decoded_preds = [index_to_class[int(p)] for p in y_pred]
submission = pd.DataFrame({"id": test_ids, "target_feature": decoded_preds})
submission = submission.sort_values("id")
submission.to_csv("final_sota_optuna_cat_lgb_submission.csv", index=False)
print("FINAL SUBMISSION SAVED: final.csv")